<a href="https://colab.research.google.com/github/GitWahome/Optimization-Methods/blob/master/Final_Proj_OM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cvxpy as cvx
from timeit import default_timer as time
import collections

Background:

My Aunt operates a trucking logistics firm in Dallas Texas.  They key to this operation is the dispatch process. 
I analyzed  how they run it and realized that they make use of a greedy approach. They receive orders from a listings board, call the list poster and negotiate a price.
There are several factors that determine whether they take a specific order. Among them are the size of the objects to be moved, the weight of the objects listed, the destination of the order, the expected roadtime which can be infered from the delivery date among others.

This in essence presents a real time Knapsack problem but there are some nuances that make it a bit of a challenge but also allow for some assumptions.

**PROBLEM (S)**:
 
 There are a number of subproblems that this set up presents all of which  can be solved using various convex and non convex approaches.

My approach to this has been to define a subproblem and then solve it using CVCPY, each time justifying why I thought he approach taken fits the context.

For a start, we will define some constants which we must take into account. 


*   A logistics company can lease trucks of any size.. 

*   Heavy Class 7 trucks have a carrying capacity of 55000 pounds.

*   Small class 2 trucks have a carrying capacity of  26000 pounds.

*   Medium class 6 trucks have a carrying capacity of 33000 pounds.


*(I'm so sorry for using pounds, but you know, the backward Americans)*

The objective is simple,  they  need to minimize the number of trucks while maximizing the weight carried by each container.


One catch is, how many goods are put in one container is dependent only on the weight of the content already in the box weight. The profits though are mainly determined by the weight of the goods being transfered hence a carrying weight capacity constraint is imposed depending on the trucks available. 

**This makes evaluation easy as we only need to track the weight of the boxes when evaluating profits. The problem thus lies in optimizing for volume while maximizing profit. I make the assumption that all vehicles are available and if not, then they can always lease a new vehicle to do the work This is where my main challenge is.**

Transport  costs are dependent on the  weight and distance that the goods need to travel. The profit is entirely dependent on the weight and distance. To extend our objective function,  **We can determine the expected profit for each item. The objective is to maximize the weight and carried by each container given the volume and weight of each item. **

NOTE: I acknowledge the essence of route optimization but that introduces a layer of complexity hence I am abstracting the routes. 

I do not have the exact number but let us assume:

1.   The maximum consumption of a heavy truck is say 55.6 per 100 Kilometers  and the minimum is 18 when it is not carrying anything(Large trucks),

2.  The maximum consumption of a medium truck is say 40.5 liters per Kilometer  and the minimum is 10 when it is not carrying anything(Medium trucks)

3.   The maximum consumption of a small truck is say 34 liters per Kilometer  and the minimum is 5 when it is not carrying anything(Small trucks)

The weight relative to the maximum carrying capacity will determine the fuel consumption. We can evaluate the expected profits by multiplying the expected fuel consumption by the cost of fuel which we i assume to be 10$ per liter. 

I set a weight check too to be returned  if the load weight is within the vehicle limit. This allows us to know which car to use since we dont want to go over the limit to avoid getting citations.

Any car can travel any distance so there are no distance constraints, if anything, distance is good since we will make more money We just have to monitor how the fuel is affected by the weight.
(In real life however, repair costs make it such that it is more efficient to use large trucks over long distances. I abstract this to keep things relatively simple)



In [0]:
def fuel_cost(weight, distance, truck_class, weight_check = True, fuel_rate=10):
  if truck_class == "H":
    max_weight, min_consumption, max_consumption= 55000, 18, 55.6
    if weight>max_weight:
        weight_check = False
  if truck_class == "M":
    max_weight, min_consumption, max_consumption= 33000, 10, 40.5
    if weight>max_weight:
      weight_check = False
  if truck_class == "S":
    max_weight, min_consumption, max_consumption= 26000, 5, 34
    if weight>max_weight:
      weight_check = False
  return (min_consumption+(weight/max_weight)*(max_consumption-min_consumption))* fuel_rate *(distance/100), weight_check

print("1KM - MAX SMALL CAPACITY: 26000")
print("MEDIUM",fuel_cost(26000, 1,"M"))
print("SMALL",fuel_cost(26000, 1,"S"))
print("\nSLIGHTLY HIGHER: 26500")
print("MEDIUM",fuel_cost(26500, 1,"M"))
print("SMALL",fuel_cost(26500, 1,"S"))

print("\n_______________________________________________________________________")


print("1KM -MAX MEDIUM CAPACITY: 26000")
print("HEAVY",fuel_cost(33000, 1,"H"))
print("MEDIUM",fuel_cost(33000, 1,"M"))
print("\nSLIGHTLY HIGHER: 28500")
print("HEAVY",fuel_cost(33500, 1,"H"))
print("MEDIUM",fuel_cost(33500,1, "M"))


1KM - MAX SMALL CAPACITY: 26000
MEDIUM (3.4030303030303033, True)
SMALL (3.4, True)

SLIGHTLY HIGHER: 26500
MEDIUM (3.449242424242424, True)
SMALL (3.455769230769231, False)

_______________________________________________________________________
1KM -MAX MEDIUM CAPACITY: 26000
HEAVY (4.056, True)
MEDIUM (4.05, True)

SLIGHTLY HIGHER: 28500
HEAVY (4.090181818181819, True)
MEDIUM (4.096212121212122, False)


We observe that smaller trucks are more efficient when the load is smaller. Large trucks with their larger capacity are more efficient for heavy loads. Any car can carry over Its capacity but then the consumption gets larger.

I have based my limits on true data but adjusted the range of consumption such that each vehicle is the most efficient at Its maximum capacity(And still is going slightly higher) but the next vehicle class becomes more efficient quickly and surpases it. 

In real life, going over the capacity entails more repair/maintenance costs thus it always makes sense to use the next vehicle class if the capacity is larger than the prescribed limit.

Details on fuel consumption by weights are provided [in this wikipedia article](https://en.wikipedia.org/wiki/Fuel_efficiency)

This sets us the problem quite nicely.  Once we receive orders which entails distances and weights, and volume, we simply have to analyze the expected profits from each item. I assume the charge is 2USD per KM. The Cost depends on the weight too. I will assume a uniform charge of 2.5 USD per lb. This thus gives us a charges function for any given load as: 2distance +2.5load. It should be higher than the fuel cost otherwise the whole operation is a waste. These values are such that we still make a profit even if we are transporting a 1 Kilo load over a distance of 1 kilometer.

Normally the price is negotiated an agreed upon thus if a customer is a noob, they can overpay but in some instances, they negotiate too much. For simpicity, I will assume an average cost. 

Quick note: In real like, we can have a minimum average and during negotiation, all we need to do is ensure the price paid by a customer does not reduce our average to below the minimum. 

So with 2$/KM in cost, we can evaluate the profit made from item trip by:

In [0]:
chargesF = lambda weight, distance: 2*weight+2.5*distance
print("SMALL",fuel_cost(1, 1,"S"))
print("MEDIUMM",fuel_cost(1, 1,"M"))
print("LARGE",fuel_cost(1, 1,"H"))
print(charges(1,1))

SMALL (0.5001115384615384, True)
MEDIUMM (1.0000924242424243, True)
LARGE (1.8000683636363635, True)
4.5


In [0]:
def profits(weights, distances):
  costs_heavy = [fuel_cost(weights[i], distances[i],"H") for i in range(len(weights))]
  costs_medium = [fuel_cost(weights[i], distances[i],"M") for i in range(len(weights))]
  costs_small = [fuel_cost(weights[i], distances[i],"S") for i in range(len(weights))]
  charges = [chargesF(weights[i],distances[i]) for i in range(len(weights))]
  
  return({"H":[(charges[i]-costs_heavy[i][0],costs_heavy[i][1]) for i in range(len(weights))], 
          "M":[(charges[i]-costs_medium[i][0],costs_medium[i][1]) for i in range(len(weights))], 
          "S":[(charges[i]-costs_small[i][0], costs_small[i][1]) for i in range(len(weights))]})

Example:

In [0]:
volumes = [800, 400, 1200, 200, 500, 220, 1000, 400]
weights = [10000, 8000, 20000, 50000, 30000, 40000, 50000, 46000]
profits(weights, distances)

{'H': [(20013.090909090908, True),
  (16061.236363636364, True),
  (39919.92727272727, True),
  (99945.63636363637, True),
  (59324.545454545456, True),
  (79955.24, True),
  (99972.81818181818, True),
  (91902.2109090909, True)],
 'M': [(20460.60606060606, True),
  (16304.242424242424, True),
  (39958.181818181816, True),
  (99937.57575757576, False),
  (59363.63636363636, True),
  (79951.66666666667, False),
  (99968.78787878787, False),
  (91889.93939393939, False)],
 'S': [(20707.69230769231, True),
  (16443.076923076922, True),
  (39972.307692307695, True),
  (99928.46153846153, False),
  (59326.92307692308, False),
  (79945.84615384616, False),
  (99964.23076923077, False),
  (91874.76923076923, False)]}

Now let us define a a solve
This should take in the item weights and volumes then give us the minimum number of containers we will need. I have included a solution decomposer that will tell us the optimum content of each container. 

Note: Sometimes weight and volume are not the only factor at play. A client may be transporting radioactive elements, or food in which case, these can only be transported together or wholly alone even if space is wasted. We skip  such cases to keep things simple. These are also easy to handle. We just assign a container to the isolated items and in foods and such, we just do the optimization for orders that are of similar type.

For now, let us solve this generally.

In [0]:
print(cvx.installed_solvers())
box_vol = 1500
box_weight = 55000

def m_solver(factor_1, factor_2, max_capacity_r1, max_capacity_r2, solver=cvx.GLPK_MI):

  max_n_boxes = len(factor_1) # real-world: heuristic?

  """ Optimization """

  # VARIABLES
  trucks_used = cvx.Variable(max_n_boxes, boolean=True)
  truck_distance = cvx.Variable(max_n_boxes)
  truck_weight = cvx.Variable(max_n_boxes)
  truck_item_map = cvx.Variable((max_n_boxes, len(factor_1)), boolean=True)

  # CONSTRAINTS
  cons = []

  # each item is shipped once
  cons.append(cvx.sum(truck_item_map, axis=0) == 1)
  # box is used when >=1 item is using it
  cons.append(trucks_used * (len(factor_1) + 1) >= cvx.sum(truck_item_map, axis=1))
  # box vol constraints
  cons.append(truck_item_map * factor_1 <= max_capacity_r1)

  # box weight constraints
  cons.append(truck_item_map * factor_2 <= max_capacity_r2)

  problem = cvx.Problem(cvx.Minimize(cvx.sum(trucks_used)), cons)
  start_t = time()
  problem.solve(solver=solver, verbose=True)
  end_t = time()
  print(f'OPTIMUM SOLUTION EVALUATED IN {end_t - start_t} SECONDS')
  print(problem.status)
  print(problem.value)
  print(truck_item_map.value)

  """ Reconstruct solution """
  n_boxes_used = int(np.round(problem.value))
  box_inds_used = np.where(np.isclose(trucks_used.value, 1.0))[0]
  truck_loads = {}
  for box in range(n_boxes_used):
      truck_loads[box]=[]
      raw = truck_item_map[box_inds_used[box]]
      items = np.where(np.isclose(raw.value, 1.0))[0]
      vol_used = 0
      weight_used = 0
      for item in items:
        truck_loads[box].append((item, (factor_1[item], factor_2[item])))
        vol_used += factor_1[item]
        weight_used += factor_2[item]
  return {"Minimum Containers":n_boxes_used,"Load List":truck_loads,"(Total Volume, Total Weight)":(vol_used, weight_used)}
solution = m_solver(factor_1= volumes,
         factor_2= weights,
         max_capacity_r1= box_vol,
         max_capacity_r2= box_weight)

print(solution)

['ECOS', 'ECOS_BB', 'CVXOPT', 'GLPK', 'GLPK_MI', 'SCS', 'OSQP']
OPTIMUM SOLUTION EVALUATED IN 13.150033977999556 SECONDS
optimal
6.0
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0.]]
{'Minimum Containers': 6, 'Load List': {0: [(2, (1200, 20000))], 1: [(7, (400, 46000))], 2: [(1, (400, 8000)), (4, (500, 30000))], 3: [(3, (200, 50000))], 4: [(6, (1000, 50000))], 5: [(0, (800, 10000)), (5, (220, 40000))]}, '(Total Volume, Total Weight)': (1020, 50000)}
['ECOS', 'ECOS_BB', 'CVXOPT', 'GLPK', 'GLPK_MI', 'SCS', 'OSQP']
OPTIMUM SOLUTION EVALUATED IN 12.504759142000694 SECONDS
optimal
6.0
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0.]]
{'Minimum Containers': 6, 'Lo

I now need a helper function to make sense of the data.
NOTE: I have incorporated this into my profit evaluator above.  

We know we need 5 trucks. The class of the track will depend on the  weight.  I am making the assumption that we can use constant container sizes. In real life, we use smaller containers on small vehicles. The volume capacity tends to vary depending on the class of the vehicle. Either way, since  the system has optimized the load for volume and weight, we can simply look at these numbers and determine which vehicles to lease.  To do this, we need to define the bounds for the different classes. We assume that each vehicle can pull one container.

For each container, we evaluate the expected profit for the three vehicle  classes, picking the vehicle class that gives us maximum profit.

We do this then output the total profit and the leased vehicles recommendation.

In [0]:
def solution_processor(solution):
  expected_profit, expected_class = [], []
  for item_id in solution['Load List'].keys():
    item, (volume, weight) = solution['Load List'][item_id][0]
    profit = profits([volume], [weight])
    prof_item = []
    truck_class ={1:"HEAVY", 2:"MID", 3:"SMALL"}
    for prof in profit.keys():
      if profit[prof][0][1]:
        prof_item.append(profit[prof][0][0])
      else:
        prof_item.append(0)
    item_profit = max(prof_item)
    truck_c = truck_class[prof_item.index(item_profit)]
    expected_profit.append(item_profit)
    expected_class.append(truck_c)
  return expected_profit, expected_class
optimum_vehicles = solution_processor(solution)
vehicles = dict(collections.Counter(optimum_vehicles[1])
for item in :vehicles.keys():
  print(f"You need {item[1]}{}{{{} "){}{} {} {}  {}  {}  

MID


Note,